In [1]:
import pandas as pd
import nltk
import pymorphy2
from string import punctuation
import re

In [2]:
path = '/home/ksenia/progas/python/int_trans/rureviews/women-clothing-accessories.3-class.balanced.csv'

загружаем наш датасет

In [3]:
df = pd.read_csv(path, sep='\t', header=0)

In [4]:
df

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative
...,...,...
89995,сделано достаточно хорошо. на ткани сделан рис...,positive
89996,Накидка шикарная. Спасибо большое провдо линяе...,positive
89997,спасибо большое ) продовца рекомендую.. заказа...,positive
89998,Очень довольна заказом! Меньше месяца в РБ. К...,positive


загружаем стоп-слова и лемматизаторы для английского и для русского (потому что в датасете перемешанные языки)

In [5]:
nltk.download('stopwords')
russian_stopwords = nltk.corpus.stopwords.words("russian")
english_stopwords = nltk.corpus.stopwords.words("english")

[nltk_data] Downloading package stopwords to /home/ksenia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
rus_lemmatizer = pymorphy2.MorphAnalyzer()

In [7]:
tokenizer = nltk.RegexpTokenizer(r"\w+")

In [8]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
eng_lemmatizer = nltk.stem.WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /home/ksenia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ksenia/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [9]:
rus_pattern = re.compile(r'[А-я]+')
eng_pattern = re.compile(r'[A-z]+')

In [10]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": nltk.corpus.wordnet.ADJ,
                "N": nltk.corpus.wordnet.NOUN,
                "V": nltk.corpus.wordnet.VERB,
                "R": nltk.corpus.wordnet.ADV}
    return tag_dict.get(tag, nltk.corpus.wordnet.NOUN)

удаляем из списка стоп-слова, которые могут влиять на таргет

In [11]:
def remove(word_list, words_to_remove):
    for word in words_to_remove:
        try:
            word_list.remove(word)
        except:
            pass

remove(russian_stopwords, ('не', 'никогда', 'хорошо', 'конечно'))

In [12]:
def preprocess_line(row):
    line = row['review'].lower()
    tokens = tokenizer.tokenize(line)
    
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    tokens = [token for token in tokens if token not in english_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    normal_forms = []
    for token in tokens:
        if rus_pattern.match(token):
            normal_form = rus_lemmatizer.parse(token)[0].normal_form
            if normal_form not in russian_stopwords:    
                normal_forms.append(normal_form)
        elif eng_pattern.match(token):
            normal_form = eng_lemmatizer.lemmatize(token, get_wordnet_pos(token))
            if normal_form not in english_stopwords:    
                normal_forms.append(normal_form)
                
    row['tokens'] = normal_forms
    row['strat_kfold'] = '{}_{}'.format(row['sentiment'], len(normal_forms))
    return row

In [13]:
from torchnlp.word_to_vector import FastText

In [14]:
ru_embeddings = FastText(language='ru')
en_embeddings = FastText(language='en')

In [15]:
from tqdm import tqdm
tqdm.pandas()

препроцессинг датасета

In [16]:
df = df.progress_apply(preprocess_line, axis=1)

100%|██████████| 90000/90000 [03:14<00:00, 462.58it/s]


In [17]:
df

,review,sentiment,tokens,strat_kfold
0,качество плохое пошив ужасный (горловина напер...,negative,"[качество, плохой, пошив, ужасный, горловина, ...",negative_21
1,"Товар отдали другому человеку, я не получила п...",negative,"[товар, отдать, человек, не, получить, посылка...",negative_9
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative,"[ужасный, синтетик, тонкий, общий, представить...",negative_20
3,"товар не пришел, продавец продлил защиту без м...",negative,"[товар, не, прийти, продавец, продлить, защита...",negative_9
4,"Кофточка голая синтетика, носить не возможно.",negative,"[кофточка, голый, синтетик, носить, не, возможно]",negative_6
...,...,...,...,...
89995,сделано достаточно хорошо. на ткани сделан рис...,positive,"[сделать, достаточно, хорошо, ткань, сделать, ...",positive_13
89996,Накидка шикарная. Спасибо большое провдо линяе...,positive,"[накидка, шикарный, спасибо, большой, провдо, ...",positive_15
89997,спасибо большое ) продовца рекомендую.. заказа...,positive,"[спасибо, большой, продовца, рекомендовать, за...",positive_6
89998,Очень довольна заказом! Меньше месяца в РБ. К...,positive,"[очень, довольный, заказ, маленький, месяц, рб...",positive_15


делаем стратифайд кфолд из 5 фолдов

In [18]:
from sklearn.model_selection import StratifiedKFold

In [19]:
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [20]:
for fold_number, (train_index, val_index) in enumerate(stratified_kfold.split(X=df.index, y=df['strat_kfold'])):
    df.loc[df.iloc[val_index].index, 'fold'] = fold_number

/home/ksenia/frow_home_projects/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


In [21]:
df

,review,sentiment,tokens,strat_kfold,fold
0,качество плохое пошив ужасный (горловина напер...,negative,"[качество, плохой, пошив, ужасный, горловина, ...",negative_21,0.0
1,"Товар отдали другому человеку, я не получила п...",negative,"[товар, отдать, человек, не, получить, посылка...",negative_9,2.0
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative,"[ужасный, синтетик, тонкий, общий, представить...",negative_20,1.0
3,"товар не пришел, продавец продлил защиту без м...",negative,"[товар, не, прийти, продавец, продлить, защита...",negative_9,4.0
4,"Кофточка голая синтетика, носить не возможно.",negative,"[кофточка, голый, синтетик, носить, не, возможно]",negative_6,2.0
...,...,...,...,...,...
89995,сделано достаточно хорошо. на ткани сделан рис...,positive,"[сделать, достаточно, хорошо, ткань, сделать, ...",positive_13,0.0
89996,Накидка шикарная. Спасибо большое провдо линяе...,positive,"[накидка, шикарный, спасибо, большой, провдо, ...",positive_15,1.0
89997,спасибо большое ) продовца рекомендую.. заказа...,positive,"[спасибо, большой, продовца, рекомендовать, за...",positive_6,3.0
89998,Очень довольна заказом! Меньше месяца в РБ. К...,positive,"[очень, довольный, заказ, маленький, месяц, рб...",positive_15,2.0


удаляем строки, которые после препроцессинга растеряли все слова (их немного, около 20)

In [22]:
df = df[df['tokens'].apply(len) != 0]

делаем датасет

In [23]:
from torch.utils.data import Dataset
import torch

In [24]:
class DatasetRetriever(Dataset):
    def __init__(self, df_dataset):
        super().__init__()
        self.df_dataset = df_dataset
        self.index_values = df_dataset.index.values
        
    def __getitem__(self, index: int):
#         print(index)
        row = self.df_dataset.loc[self.index_values[index]]
        
        tensor = torch.zeros([100, 300], dtype=torch.float32)
        
        for i, token in enumerate(row['tokens']):
            if i >= 100:
                break
            
            if rus_pattern.match(token):
                tensor[i, :] = ru_embeddings[token]
            elif eng_pattern.match(token):
                tensor[i, :] = en_embeddings[token]
        
        label = 0
        if row['sentiment'] == 'neautral':
            label = 1
        elif row['sentiment'] == 'positive':
            label = 2
            
        return tensor, label
    
    def __len__(self):
        return self.index_values.shape[0]

In [25]:
fold_number = 0

In [26]:
train_dataset = DatasetRetriever(df[df['fold'] != fold_number])
val_dataset = DatasetRetriever(df[df['fold'] == fold_number])

проверяем, что датасет построился корректно

In [27]:
data = train_dataset[0]

In [28]:
data

(tensor([[ 0.5515, -0.0717, -0.1112,  ...,  0.2617,  0.4814, -0.4449],
         [ 0.1840,  0.2454, -0.2898,  ..., -0.1282, -0.2901,  0.4213],
         [-0.0395, -0.2272,  0.1935,  ..., -0.0100, -0.1458,  0.0633],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]),
 0)

создаем переменные для нейронки

In [29]:
device = torch.device('cuda:0')
batch_size = 128
num_workers = 8

In [30]:
from torch.utils.data.sampler import RandomSampler, SequentialSampler

In [31]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    sampler=RandomSampler(train_dataset),
    pin_memory=False,
    drop_last=True,
    num_workers=num_workers,
    collate_fn=collate_fn,
)

val_loader = torch.utils.data.DataLoader(
    val_dataset, 
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=False,
    sampler=SequentialSampler(val_dataset),
    pin_memory=False,
    collate_fn=collate_fn,
)

In [35]:
class Model(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super(Model, self).__init__()
                
        dim1, dim2  = 8, 16
        
        self.cnn_layer1 = torch.nn.Sequential(
            torch.nn.Conv1d(dim_in, dim1, kernel_size=1, stride=1, padding=0, bias=False),
            torch.nn.BatchNorm1d(num_features=dim1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv1d(dim1, dim2, kernel_size=1, stride=1, padding=0, bias=False),
            torch.nn.BatchNorm1d(num_features=dim2),
            torch.nn.ReLU(inplace=True),
        )
        self.cnn_layer2 = torch.nn.Sequential(
            torch.nn.Conv1d(dim_in, dim1, kernel_size=3, stride=1, padding=1, bias=False),
            torch.nn.BatchNorm1d(num_features=dim1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv1d(dim1, dim2, kernel_size=3, stride=1, padding=1, bias=False),
            torch.nn.BatchNorm1d(num_features=dim2),
            torch.nn.ReLU(inplace=True),
        )
        self.cnn_layer3 = torch.nn.Sequential(
            torch.nn.Conv1d(dim_in, dim1, kernel_size=5, stride=1, padding=2, bias=False),
            torch.nn.BatchNorm1d(num_features=dim1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv1d(dim1, dim2, kernel_size=5, stride=1, padding=2, bias=False),
            torch.nn.BatchNorm1d(num_features=dim2),
            torch.nn.ReLU(inplace=True),
        )
        self.cnn_layer4 = torch.nn.Sequential(
            torch.nn.Conv1d(dim_in, dim1, kernel_size=7, stride=1, padding=3, bias=False),
            torch.nn.BatchNorm1d(num_features=dim1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv1d(dim1, dim2, kernel_size=7, stride=1, padding=3, bias=False),
            torch.nn.BatchNorm1d(num_features=dim2),
            torch.nn.ReLU(inplace=True),
        )
        self.cnn_layer5 = torch.nn.Sequential(
            torch.nn.Conv1d(dim_in, dim1, kernel_size=9, stride=1, padding=4, bias=False),
            torch.nn.BatchNorm1d(num_features=dim1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv1d(dim1, dim2, kernel_size=9, stride=1, padding=4, bias=False),
            torch.nn.BatchNorm1d(num_features=dim2),
            torch.nn.ReLU(inplace=True),
        )
        
        self.max_pool1 = torch.nn.MaxPool1d(kernel_size=2)
        self.max_pool2 = torch.nn.MaxPool1d(kernel_size=2)
        self.max_pool3 = torch.nn.MaxPool1d(kernel_size=2)
        self.max_pool4 = torch.nn.MaxPool1d(kernel_size=2)
        self.max_pool5 = torch.nn.MaxPool1d(kernel_size=2)
        
        self.linear1 = torch.nn.Linear(12000, dim_out)
        
    def forward(self, matrix):
        x1 = self.cnn_layer1(matrix)
        x2 = self.cnn_layer2(matrix)
        x3 = self.cnn_layer3(matrix)
        x4 = self.cnn_layer4(matrix)
        x5 = self.cnn_layer5(matrix)
        
        x1 = self.max_pool1(x1)
        x2 = self.max_pool2(x2)
        x3 = self.max_pool3(x3)
        x4 = self.max_pool4(x4)
        x5 = self.max_pool5(x5)
        
        x1 = torch.flatten(x1, 1)
        x2 = torch.flatten(x2, 1)
        x3 = torch.flatten(x3, 1)
        x4 = torch.flatten(x4, 1)
        x5 = torch.flatten(x5, 1)
        
        x6 = torch.cat((x1, x2, x3, x4, x5), dim=1)
        x7 = self.linear1(x6)
        
        return x7

In [44]:
model = Model(100, 3)

проверяем, что батч запихивается в сеть

In [45]:
input_tensor = torch.stack((data[0], data[0], data[0]))
print(input_tensor.shape)
output = model(input_tensor)

torch.Size([3, 100, 300])


In [46]:
output.shape

torch.Size([3, 3])

In [47]:
output

tensor([[ 0.4321, -0.0324,  0.3626],
        [ 0.4321, -0.0324,  0.3626],
        [ 0.4321, -0.0324,  0.3626]], grad_fn=<AddmmBackward>)

дальше будет обучение 

In [48]:
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [49]:
from sklearn.metrics import confusion_matrix, classification_report

In [50]:
for epoch in range(10):
    running_loss = 0.0
    
    # train
    
    for batch_idx, (matrices, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        
        batch_tensor = torch.stack(matrices).to(device)
        labels_tensor = torch.tensor(labels).to(device)
        
        outputs = model(batch_tensor)
        loss = criterion(outputs, labels_tensor)
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
        
        if batch_idx % 100 == 0:
            print(batch_idx, running_loss/100, loss.item())
            running_loss = 0.0
            
    running_loss = 0.0
    accuracy = 0
    pred_array = []
    label_array = []

    #test
    
    with torch.no_grad():
        for i, (matrices, labels) in enumerate(val_loader):
            batch_tensor = torch.stack(matrices).to(device)
            labels_tensor = torch.tensor(labels).to(device)

            outputs = model(batch_tensor)
            loss = criterion(outputs, labels_tensor)
            running_loss += loss.item()

            pred_array += torch.argmax(outputs, dim=1).tolist()
            label_array += labels_tensor.tolist()

            accurate_predicted = torch.sum(torch.argmax(outputs, dim=1) == labels_tensor).item()
            accuracy += accurate_predicted

    print('----------------------------------------------')
    print('loss:  ', running_loss/len(val_loader))
    print('accuracy:  ', accuracy / len(val_loader))
    print(confusion_matrix(label_array, pred_array))
    print(classification_report(label_array, pred_array))
    print('----------------------------------------------\n\n\n')
            

0 0.011453495025634766 1.1453495025634766
100 0.9870296752452851 0.821228563785553
200 0.7736449813842774 0.812629759311676
300 0.7386571186780929 0.6705959439277649
400 0.7328903269767761 1.0365992784500122
500 0.7220871156454086 0.628259539604187
----------------------------------------------
loss:   0.7107365685449519
accuracy:   87.37323943661971
[[3192 2635  204]
 [ 807 4551  654]
 [ 131 1224 4664]]
              precision    recall  f1-score   support

           0       0.77      0.53      0.63      6031
           1       0.54      0.76      0.63      6012
           2       0.84      0.77      0.81      6019

   micro avg       0.69      0.69      0.69     18062
   macro avg       0.72      0.69      0.69     18062
weighted avg       0.72      0.69      0.69     18062

----------------------------------------------



0 0.006469257473945618 0.6469257473945618
100 0.6540451401472092 0.6819949150085449
200 0.6709786087274552 0.6410307288169861
300 0.6769006302952767 0.6070649027